# Motivation
***
This notebook will discuss many different standard algorithms for implementing dimensionality reduction using the scikit-learn library.

Dimensionality reduction can be used to counteract the curse of dimensionality reduction, which is when machine learning algorithms become inefficient due to an extremely large feature space. 

Dimensionality reduction algorithms are used to project high-dimensional data to a low-dimensional space while retaining as much of the salient information as possible while removing redundant information. This enables supervised machine learning algorithms to identify interesting patterns more effectively because much of the noise will have been reduced. 

# The Data
***
We will be using the MNIST Digits database. One may acquire the pickled version [here](http://deeplearning.net/tutorial/gettingstarted.html).

This dataset is divided into a training set with 50,000 examples, a validation set with 10,000 examples and a test set with 10,000 examples. The data consist of 28x28 pixel images of handwritten digits. Each row of the data will be a flattened (28x28=784) dimensional vector. The values of the components of this vector are intensity values in the range zero to one, where zero is black and one is white. The labels are digits zero through nine. 

Standard Imports:

In [1]:
import numpy as np
import pandas as pd
import os, time
import pickle, gzip 
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
import matplotlib as mpl

%matplotlib inline

from sklearn import preprocessing as pp
from scipy.stats import pearsonr
from numpy.testing import assert_array_almost_equal
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import log_loss, precision_recall_curve, average_precision_score, roc_curve, auc, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb

Now we will load the data